In [ ]:
import argparse
import time
 
import numpy as np
from PIL import Image
#import tensorflow as tf # TF1
import tflite_runtime.interpreter as tflite
import cv2
import os 

In [ ]:
def load_labels(filename):
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

In [ ]:
model_file = '/home/ys/tensorflow_test/test/tflite_python_test/float16_model.tflite'
#model_file = '/home/ys/tensorflow_test/test/tflite_python_test/tmp/mobilenet_v1_1.0_224.tflite'
image = '../image1.jpg'

In [ ]:
def detect_tflite(model_file, image):
    #interpreter = tf.lite.Interpreter(model_path=model_file)
    interpreter = tflite.Interpreter(model_path=model_file)
    interpreter.allocate_tensors()
    print('ok')
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # check the type of the input tensor
    #floating_model = input_details[0]['dtype'] == np.float32

     # NxHxWxC, H:1, W:2
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    #img = Image.open(image_path).resize((width, height))
    img = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    # add N dim
    input_data = np.expand_dims(img, axis=0)

    #if floating_model:
    #    input_data = (np.float32(input_data) - args.input_mean) / args.input_std

    input_data = input_data.astype(np.float32) / 255
    print(input_data.dtype)
    input_shape = input_details[0]['shape']
    #input_data = np.array(np.random.random_sample(input_shape), dtype=np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    #time_start = time.time()
    interpreter.invoke()
    #time_end   = time.time()
    #print(output_details)
    # 函数`get_tensor()`返回该tensor数据的一份拷贝
    # 使用`tensor()`返回该tensor的指针
    detection_boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    detection_classes = interpreter.get_tensor(output_details[1]['index'])[0]
    detection_scores = interpreter.get_tensor(output_details[2]['index'])[0]
    num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
    #print(detection_boxes,detection_classes,detection_scores,num_detections)
    #print('time cost',(time_end - time_start) * 1000)
    return(detection_boxes,detection_classes,detection_scores,num_detections)

In [ ]:
label_dict = {0:'ball', 1:'goal', 2:'robot'}
def plot_box(img, detection_boxes, detection_classes, detection_scores, num_detections):
    Height = img.shape[0]
    Width = img.shape[1]
    bboxes = []
    cls_list = []
    for i in range(int(num_detections)):
    
        if detection_scores[i] < 0.3:
            continue
        else:
            y1 = int(detection_boxes[i][0] * Height)
            x1 = int(detection_boxes[i][1] * Width)
            y2 = int(detection_boxes[i][2] * Height)
            x2 = int(detection_boxes[i][3] * Width)
            bboxes.append([x1, y1, x2, y2])
            cls = int(detection_classes[i])
            if cls in cls_list:
                continue
            else:
                cls_list.append(cls)
            cls_name = label_dict[cls]
            print(cls_name)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, cls_name+str(detection_scores[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,
                                1.0, (255, 255, 255), 1, cv2.LINE_AA, False)
    cv2.imshow("figure", img)
    cv2.waitKey()
    if 0xFF == ord('q'):
        cv2.destroyAllWindows() 

In [ ]:

img_path_dir = '/home/ys/image/2020_12_24_09h41m21s'
img_path_lst = os.listdir(img_path_dir)

for img_path in img_path_lst:
    img = cv2.imread(os.path.join(img_path_dir, img_path))
    cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    print(img.shape)
    time_start=time.time()
    #ret, frame = cap.read()
    detection_boxes,detection_classes,detection_scores,num_detections = detect_tflite(model_file, img)
    
    time_end=time.time()
    print('time cost',(time_end-time_start)*1000,'ms')
    
    plot_box(img, detection_boxes, detection_classes, detection_scores,num_detections)
if cv2.waitKey(25) & 0xFF == ord('q'):
    cv2.destroyAllWindows()